In [1]:
import numpy as np
import random
import math
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd


### Funções auxiliares: avaliação de conflitos e visualização da solução
Define a função `h(state)` que calcula o número de conflitos entre rainhas, e `plot_solution()` para desenhar o tabuleiro.


In [7]:
def conflict(row1, col1, row2, col2):
    return (row1 == row2 or
            col1 == col2 or
            row1 - col1 == row2 - col2 or
            row1 + col1 == row2 + col2)

def h(state):
    conflicts = 0
    N = len(state)
    for i in range(N):
        for j in range(i + 1, N):
            if conflict(state[i], i, state[j], j):
                conflicts += 1
    return conflicts

def plot_solution(solution, N):
    fig = plt.figure()
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim((0, N))
    ax.set_ylim((0, N))
    for i, row in enumerate(solution):
        ax.add_patch(patches.Rectangle((i, row), 1, 1, color='black'))
    plt.gca().invert_yaxis()
    plt.show()


### Função principal: Simulated Annealing
Implementa o algoritmo de Simulated Annealing, que aceita movimentos piores com uma certa probabilidade, reduzindo a chance ao longo do tempo (temperatura decrescente).


In [8]:
def simulated_annealing(N, runs=5, T_start=100, T_min=1e-3, alpha=0.99, max_iter=1000):
    results = []

    for run in range(1, runs + 1):
        start = time.time()

        current = [random.randint(0, N - 1) for _ in range(N)]
        current_cost = h(current)
        T = T_start
        best = current[:]
        best_cost = current_cost

        for _ in range(max_iter):
            if current_cost == 0 or T < T_min:
                break

            # Escolhe uma coluna e move rainha para linha diferente
            neighbor = current[:]
            col = random.randint(0, N - 1)
            new_row = random.choice([r for r in range(N) if r != current[col]])
            neighbor[col] = new_row
            neighbor_cost = h(neighbor)

            delta = neighbor_cost - current_cost
            if delta < 0 or random.random() < math.exp(-delta / T):
                current = neighbor
                current_cost = neighbor_cost
                if current_cost < best_cost:
                    best = current[:]
                    best_cost = current_cost

            T *= alpha

        end = time.time()
        solved = best_cost == 0
        exec_time = end - start

        results.append({
            "N": N,
            "Run": run,
            "Solved": solved,
            "Fitness": best_cost,
            "Time": exec_time,
            "Solution": best
        })

    return results


### Execução do Simulated Annealing para diferentes tamanhos de tabuleiro
Executa o algoritmo 5 vezes para cada valor de N, exibe os resultados e plota a melhor solução, se encontrada.


In [9]:
all_sa_results = []

for N in [32, 64, 128]:
    print(f"\nSimulated Annealing para N={N}")
    results = simulated_annealing(N)

    for r in results:
        print(f"Run {r['Run']}: Solved={r['Solved']} | Fitness={r['Fitness']} | Time={r['Time']:.2f}s")
        all_sa_results.append(r)

    best_result = min(results, key=lambda x: x["Fitness"])
    if best_result["Solved"]:
        print("➡️ Solução encontrada:")
        plot_solution(best_result["Solution"], N)
    else:
        print(f"⚠️ Melhor solução tinha {best_result['Fitness']} conflitos")



Simulated Annealing para N=32
Run 1: Solved=False | Fitness=4 | Time=0.12s
Run 2: Solved=False | Fitness=6 | Time=0.16s
Run 3: Solved=False | Fitness=6 | Time=0.23s
Run 4: Solved=False | Fitness=2 | Time=0.22s
Run 5: Solved=False | Fitness=5 | Time=0.21s
⚠️ Melhor solução tinha 2 conflitos

Simulated Annealing para N=64
Run 1: Solved=False | Fitness=12 | Time=0.53s
Run 2: Solved=False | Fitness=16 | Time=0.61s
Run 3: Solved=False | Fitness=16 | Time=0.55s
Run 4: Solved=False | Fitness=15 | Time=0.43s
Run 5: Solved=False | Fitness=14 | Time=0.49s
⚠️ Melhor solução tinha 12 conflitos

Simulated Annealing para N=128
Run 1: Solved=False | Fitness=43 | Time=1.92s
Run 2: Solved=False | Fitness=41 | Time=1.82s
Run 3: Solved=False | Fitness=38 | Time=1.81s
Run 4: Solved=False | Fitness=46 | Time=1.65s
Run 5: Solved=False | Fitness=40 | Time=1.69s
⚠️ Melhor solução tinha 38 conflitos


In [10]:
df_sa = pd.DataFrame([{
    "N": r["N"],
    "Run": r["Run"],
    "Solved": r["Solved"],
    "Fitness": r["Fitness"],
    "Time": r["Time"]
} for r in all_sa_results])

df_sa


,N,Run,Solved,Fitness,Time
0,32,1,False,4,0.123165
1,32,2,False,6,0.156714
2,32,3,False,6,0.233941
3,32,4,False,2,0.215304
4,32,5,False,5,0.214203
5,64,1,False,12,0.531750
6,64,2,False,16,0.606842
7,64,3,False,16,0.545943
8,64,4,False,15,0.429798
9,64,5,False,14,0.486208
